In [1]:
import matplotlib.pyplot as plt
from metpy.units import units
import numpy as np
import os
import pandas as pd
from pathlib import Path
import pygrib
from statisticplot import make_map
import xarray

In [2]:
from awips.dataaccess import DataAccessLayer
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [3]:
# Unidata's cloud EDEX instance is used in this example
edex_url = "edex-cloud.unidata.ucar.edu"
DataAccessLayer.changeEDEXHost(edex_url)

In [4]:
dataTypes = DataAccessLayer.getSupportedDatatypes()
dataTypes.sort()
list(dataTypes)

['acars',
 'airep',
 'binlightning',
 'bufrmosAVN',
 'bufrmosETA',
 'bufrmosGFS',
 'bufrmosHPC',
 'bufrmosLAMP',
 'bufrmosMRF',
 'bufrua',
 'climate',
 'common_obs_spatial',
 'gfe',
 'gfeEditArea',
 'grid',
 'maps',
 'modelsounding',
 'obs',
 'pirep',
 'practicewarning',
 'profiler',
 'radar',
 'radar_spatial',
 'satellite',
 'sfcobs',
 'topo',
 'warning']

In [5]:
# Create a request for data type grid
grid_request = DataAccessLayer.newDataRequest()
grid_request.setDatatype("grid")

In [6]:
# Grid Locations
grid_locations = DataAccessLayer.getAvailableLocationNames(grid_request)
grid_locations.sort()
list(grid_locations)

['AUTOSPE',
 'ESTOFS',
 'ETSS',
 'FFG-ALR',
 'FFG-FWR',
 'FFG-KRF',
 'FFG-MSR',
 'FFG-ORN',
 'FFG-PTR',
 'FFG-RHA',
 'FFG-RSA',
 'FFG-STR',
 'FFG-TAR',
 'FFG-TIR',
 'FFG-TUA',
 'GFS1p0',
 'GFS20',
 'GlobalWave',
 'HFR-EAST_6KM',
 'HFR-EAST_PR_6KM',
 'HFR-US_EAST_DELAWARE_1KM',
 'HFR-US_EAST_FLORIDA_2KM',
 'HFR-US_EAST_NORTH_2KM',
 'HFR-US_EAST_SOUTH_2KM',
 'HFR-US_EAST_VIRGINIA_1KM',
 'HFR-US_HAWAII_1KM',
 'HFR-US_HAWAII_2KM',
 'HFR-US_HAWAII_6KM',
 'HFR-US_WEST_500M',
 'HFR-US_WEST_CENCAL_2KM',
 'HFR-US_WEST_LOSANGELES_1KM',
 'HFR-US_WEST_LOSOSOS_1KM',
 'HFR-US_WEST_NORTH_2KM',
 'HFR-US_WEST_SANFRAN_1KM',
 'HFR-US_WEST_SOCAL_2KM',
 'HFR-US_WEST_WASHINGTON_1KM',
 'HFR-WEST_6KM',
 'HPCGuide',
 'HPCqpfNDFD',
 'HRRR',
 'HRRR-Smoke',
 'LAMP2p5',
 'MRMS_0500',
 'MRMS_1000',
 'MRMS_AK_1000',
 'MRMS_CA_0500',
 'MRMS_CA_1000',
 'MRMS_GU_0500',
 'MRMS_HI_0500',
 'NAM12',
 'NAM40',
 'NOHRSC-SNOW',
 'NationalBlend',
 'RAP13',
 'RTMA',
 'RTOFS-Now-WestAtl',
 'RTOFS-Now-WestConus',
 'RTOFS-WestAtl'

In [7]:
# Pick a model and set the location for the grid request -- we'll be using the Global Forecast System (GFS)
grid_request.setLocationNames("HRRR")
grid_params = DataAccessLayer.getAvailableParameters(grid_request)
grid_params.sort()
list(grid_params)

['36SHRMi',
 '50dbzZ',
 'AV',
 'Along',
 'AppT',
 'BLI',
 'BRN',
 'BRNEHIi',
 'BRNSHR',
 'BRNmag',
 'BRNvec',
 'BdEPT06',
 'BlkMag',
 'BlkShr',
 'CAPE',
 'CCP',
 'CFRZR',
 'CICEP',
 'CIn',
 'CPOFP',
 'CPr',
 'CPrD',
 'CRAIN',
 'CSNOW',
 'CURU',
 'CXR',
 'CapeStk',
 'Corf',
 'CorfF',
 'CorfFM',
 'CorfM',
 'CritT1',
 'CumNrm',
 'CumShr',
 'DivF',
 'DivFn',
 'DivFs',
 'DpD',
 'DpT',
 'EHI',
 'EHI01',
 'EHIi',
 'EPT',
 'EPTA',
 'EPTC',
 'EPTGrd',
 'EPTGrdM',
 'EPTs',
 'EPVg',
 'EPVs',
 'EPVt1',
 'EPVt2',
 'ESP',
 'ESP2',
 'FVecs',
 'FeatMot',
 'FnVecs',
 'FsVecs',
 'Fzra1',
 'Fzra2',
 'GH',
 'GHxSM',
 'GHxSM2',
 'GVV',
 'GVV1hr',
 'Gust',
 'HCDC',
 'HI4',
 'HIdx',
 'HPBL',
 'Heli',
 'HeliC',
 'INV',
 'IPLayer',
 'Into',
 'KI',
 'L-I',
 'LCDC',
 'LIsfc2x',
 'LM5',
 'LM6',
 'LTNG',
 'MAXDVV',
 'MAXDVV1hr',
 'MAXREF',
 'MAXREF1hr',
 'MAXUPHL',
 'MAXUVV',
 'MAXUVV1hr',
 'MAdv',
 'MCDC',
 'MCon',
 'MCon2',
 'MLLCL',
 'MMP',
 'MSFDi',
 'MSFi',
 'MSFmi',
 'MSG',
 'MTV',
 'MXDVV',
 'MXREF',
 'MXUP

In [8]:
parameters = ["LTNG","REFC"]
grid_request.setParameters(*parameters)
grid_levels = DataAccessLayer.getAvailableLevels(grid_request)
for lvl in grid_levels:
    print(lvl)

0.0EA


In [9]:
grid_request.getParameters()

['LTNG', 'REFC']

In [10]:
grid_cycles = DataAccessLayer.getAvailableTimes(grid_request, True)
grid_cycles

[<DataTime instance: 2023-06-13 19:00:00 >,
 <DataTime instance: 2023-06-13 20:00:00 >,
 <DataTime instance: 2023-06-13 21:00:00 >,
 <DataTime instance: 2023-06-13 22:00:00 >]

In [11]:
# Available grid times
grid_times = DataAccessLayer.getAvailableTimes(grid_request)
## Using -1 in an array will access the last element of the array 
##  (using -2 will access the second last element, and so on)
grid_fcstRun = DataAccessLayer.getForecastRun(grid_cycles[-2], grid_times)

## print out the time instances
times = []
for fcst in grid_fcstRun:
    print(fcst.getRefTime(), ':', fcst.getFcstTime()/3600, 'hours')

2023-06-13 21:00:00.000 : 1.0 hours
2023-06-13 21:00:00.000 : 2.0 hours
2023-06-13 21:00:00.000 : 3.0 hours
2023-06-13 21:00:00.000 : 4.0 hours
2023-06-13 21:00:00.000 : 5.0 hours
2023-06-13 21:00:00.000 : 6.0 hours
2023-06-13 21:00:00.000 : 7.0 hours
2023-06-13 21:00:00.000 : 9.0 hours
2023-06-13 21:00:00.000 : 10.0 hours
2023-06-13 21:00:00.000 : 11.0 hours
2023-06-13 21:00:00.000 : 12.0 hours
2023-06-13 21:00:00.000 : 13.0 hours
2023-06-13 21:00:00.000 : 14.0 hours
2023-06-13 21:00:00.000 : 15.0 hours
2023-06-13 21:00:00.000 : 16.0 hours
2023-06-13 21:00:00.000 : 17.0 hours
2023-06-13 21:00:00.000 : 18.0 hours


In [12]:
## Grid Data
get_fhr = 8
grid_response = DataAccessLayer.getGridData(grid_request, [grid_fcstRun[get_fhr]])

In [13]:
for grid in grid_response:
    grid_data = grid.getRawData()
    grid_data = np.ma.array(grid_data, mask=grid_data==-999999.)
    lons, lats = grid.getLatLonCoords()
    print('Time :', grid.getDataTime(), "+", grid.getDataTime().getFcstTime()/3600, 'hours')
    ## Take a look at some information in our data
    print('Parm :', str(grid.getParameter()))
    print('Unit :', str(grid.getUnit()))

valid_time = pd.to_datetime(str(grid.getDataTime())) + pd.Timedelta(seconds=grid.getDataTime().getFcstTime())
print('Model:', str(grid.getLocationName()))
print(grid_data.shape)

Time : 2023-06-13 21:00:00 + 10.0 hours
Parm : LTNG
Unit : count
Model: HRRR
(1377, 2145)


In [14]:
ltng, refc = (np.ma.array(grid.getRawData(), mask=grid.getRawData()==-999999.) for grid in grid_response)

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
bbox = [lons.min(), lons.max(), lats.min(), lats.max()]
ltng.mask = ltng <= 0.01

In [ ]:
grid_response

In [ ]:
cmap = plt.get_cmap('rainbow')
bbox = [-103, -97, 40, 43]
fig, ax = make_map(bbox=None)
cs = ax.pcolormesh(lons, lats, ltng, transform=ccrs.PlateCarree(), 
                   cmap=cmap)

cbar = fig.colorbar(cs, shrink=0.7, orientation='vertical')
cbar.set_label(grid_response[parameters.index("LTNG")].getLevel() + " " \
               + grid_response[parameters.index("LTNG")].getParameter())
rc = ax.contour(lons, lats, refc, levels=[30,45,60], transform=ccrs.PlateCarree())
cb2 = fig.colorbar(rc, shrink=0.7, orientation='vertical')
cb2.set_label(grid_response[parameters.index("REFC")].getLevel() + " " \
              + grid_response[parameters.index("REFC")].getParameter())
ax.set_title(grid_response[0].getLocationName() + " init " \
             + str(grid.getDataTime().getRefTime()) + " valid " + valid_time.strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
fhr = grid.getDataTime()
fhr = pd.Timedelta(seconds=fhr.getFcstTime())/pd.Timedelta(hours=1)
fhr = int(fhr)

In [ ]:
itime=grid.getDataTime()
itime = pd.to_datetime(str(itime))
# We only pull 0,6,12,18z initialization times
itime = itime.floor(pd.Timedelta(hours=6))

In [ ]:
hh = itime.hour
idir = Path("HRRR/HRRR") / itime.strftime("%Y%m%d%H")
ds = xarray.open_dataset(idir/f"hrrr.t{hh}z.wrfsfcf{fhr:02d}.grib2",
                         filter_by_keys={'typeOfLevel': 'atmosphere', 'stepType': 'instant'},
                         backend_kwargs={"indexpath": os.getenv('TMPDIR') + "/{short_hash}.idx"}
)

In [ ]:
ds.ltng.plot(vmin=-1)

In [ ]:
ds

In [ ]:
ds